## Internet data

Для скачивания HTML-страниц в питоне есть специальный модуль `urllib.request`. У него есть альтернативы: https://stackoverflow.com/questions/7191337/python-better-network-api-than-urllib

На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер запрос ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ..."). Чтобы получить страницу через питон, нужно сформировать запрос на сервер так же, как это делает браузер:

In [18]:
import urllib.request #importing module
import re
req = urllib.request.Request('https://habr.com/')
with urllib.request.urlopen(req) as response:
   html = response.read().decode('utf-8')

В переменной `req` у нас как раз находится запрос. Функция `urlopen` получает ответ сервера и скачивает страницу по ссылке https://habr.com/ в переменную `response`. `response` ведет себя как файл: например мы можем прочитать его содержимое с помощью `.read()` в другую переменную. Вот так просто мы сохранили код страницы в переменной `html`. Убедимся, что в там лежит html-код:

In [19]:
print(html[:210])

<!DOCTYPE html>
<html lang="ru" class="no-js">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta content='width=1024' name='viewport'>
<title>Лучшие публикации за сутки / 


Иногда сайт блокирует запросы, если их посылает не настоящий браузер с пользователем, а какой-то бот (например, так делает Гугл или Википедия). Иногда сайты присылают разные версии страниц, разным браузерам.
По этим причинам полезно бывает писать скрипт, который умеет притворяться то одним, то другим браузером. Когда мы пытаемся получить страницу с помощью `urllib`, наш код по умолчанию честно сообщает серверу, что он является программой на питоне. Он говорит что-то вроде "Привет, я Python-urllib/3.5". Но можно, например, представиться Мозиллой:

In [20]:
url = 'https://habr.com/'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' 
req = urllib.request.Request(url, headers={'User-Agent':user_agent})  

with urllib.request.urlopen(req) as response:
   html = response.read().decode('utf-8')

Код страницы у нас уже есть, но как из него что-то вытащить. Для начала нужно посмотреть в исходник <view-source:https://habr.com/> и заметить, что заголовки хранятся в тэге **h2** с классом **post__title**. Заголовок выглядит примерно так:

<h2 class="post__title">
    <a href="https://habr.com/company/google/blog/425279/" class="post__title_link">Как собеседует Google: чему быть, чего не миновать</a>
  </h2>

Чтобы вытащить все такие заголовки, воспользуемся регулярным выражением.

In [21]:
regPostTitle = re.compile('<h2 class="post__title">.*?</h2>', flags= re.DOTALL)
titles = regPostTitle.findall(html)

Посмотрим, сколько там заголовков. И взглянем, например, на первые три.

In [22]:
print(len(titles))

20


In [26]:
print(titles[:3])

['<h2 class="post__title">\n    <a href="https://habr.com/company/badoo/blog/425325/" class="post__title_link">Интерпретаторы байт-кодов своими руками</a>\n  </h2>', '<h2 class="post__title">\n    <a href="https://habr.com/company/mailru/blog/423409/" class="post__title_link">Как сделать самому Луноход</a>\n  </h2>', '<h2 class="post__title">\n    <a href="https://habr.com/post/425359/" class="post__title_link">Китайцы использовали микрочип, чтобы контролировать американские компьютеры</a>\n  </h2>']


Теперь давайте очистим заголовки от лишних переносов строк, лишних тэгов и распечатаем их подряд.

In [24]:
new_titles = []
regTag = re.compile('<.*?>', re.DOTALL)
regSpace = re.compile('\s{2,}', re.DOTALL)
for t in titles:
    clean_t = regSpace.sub("", t)
    clean_t = regTag.sub("", clean_t)
    new_titles.append(clean_t)
for t in new_titles:
    print(t)

Интерпретаторы байт-кодов своими руками
Как сделать самому Луноход
Китайцы использовали микрочип, чтобы контролировать американские компьютеры
286 и сеть
Scrum is dead
КТРУ (Каталог товаров, работ, услуг) или смерть IT госзакупкам
Firecore — нескучная игра на AVR
Измерение времени с наносекундной точностью
Что такое ZFS? И почему люди от неё без ума?
﻿Perl 5: как в макросах ошибки прятались
10 физических фактов, которые вы должны были узнать в школе, но, возможно, не узнали
Бэкендеры всех стран объединились. Рассказываем, как прошел Backend United #2
Автор книги «Ведьмак» потребовал у CD Projekt Red минимум $16 млн
Алиса поможет разработчикам найти объекты в запросах пользователей. NER в Диалогах
Так ли хороши джуны?
Видео докладов с Deerploy DevOps MeetUp
Люди-батарейки: теоретический анализ наногенераторов на базе трибоэлектрического эффекта
Конкурс, новые курсы и фичи (осенние новости Хекслета)
Обновление до Windows 1809 (иногда) уничтожает все файлы в профиле
Как добавить остроты л

Ну и осталось убрать некрасивые кусочки html, а именно заменить специальные html-последовательности nbsp и rarr на стрелочку, например.

In [16]:
for t in new_titles:
    print(t.replace("&nbsp;&rarr;", " -> "))

Интерпретаторы байт-кодов своими руками
Как сделать самому Луноход
Китайцы использовали микрочип, чтобы контролировать американские компьютеры
286 и сеть
Scrum is dead
КТРУ (Каталог товаров, работ, услуг) или смерть IT госзакупкам
Firecore — нескучная игра на AVR
Измерение времени с наносекундной точностью
Что такое ZFS? И почему люди от неё без ума?
﻿Perl 5: как в макросах ошибки прятались
10 физических фактов, которые вы должны были узнать в школе, но, возможно, не узнали
Бэкендеры всех стран объединились. Рассказываем, как прошел Backend United #2
Автор книги «Ведьмак» потребовал у CD Projekt Red минимум $16 млн
Алиса поможет разработчикам найти объекты в запросах пользователей. NER в Диалогах
Так ли хороши джуны?
Видео докладов с Deerploy DevOps MeetUp
Люди-батарейки: теоретический анализ наногенераторов на базе трибоэлектрического эффекта
Конкурс, новые курсы и фичи (осенние новости Хекслета)
Обновление до Windows 1809 (иногда) уничтожает все файлы в профиле
Как добавить остроты л

+ Что такое `re.compile`?

Грубо говоря, `compile()` позволяет запомнить регулярное выражение и использовать его несколько раз. Суть в том, что перед тем как прогнать регулярку через строку, питон должен ее "скомпилировать" - превратить строку с регулярным выражением в специальный объект.
Строчка `re.search(..., ...)` сначала компилирует регулярное выражение, а потом выполняет поиск. Если нужно поискать что-то один раз, то такая строчка очень удобна. А если нужно поискать что-то много раз, то получится что одно и то же выражение мы компилируем много раз. А хочется один раз скомпилировать и потом много раз пользоваться. Поэтому пишут так:

In [25]:
text = 'тут текст, внутри которого мы что-то ищем'
regName = re.compile('тут регулярное выражение') # скомпилировали
toSearch = regName.search(text) # теперь можно искать в тексте
toFindAll = regName.findall(text)  # можно использовать скомпилированное выражение много раз
toSub = regName.sub('на.что.заменить', text) # и так тоже можно использовать

+ Что делает `regName.sub(..., ...)`?

Выражение `regName.sub('на_что_заменить', text)` значит: возьми скомпилированное выражение из переменной `regName`, и замени все, что соответствует этому выражению в переменной `text`, на строку 'на\_что\_заменить'. Если первый аргумент в этом случае - пустая строка, то все найденные регуляркой куски заменятся на пустую строку, короче говоря, удалятся.

+ Что такое `re.DOTALL`?

Обычно точка в регулярном выражении означает любой символ КРОМЕ символа новой строки. Чтобы изменить такое поведение, в компиляцию регулярки можно добавить параметры-флаги вот так: `flags = re.DOTALL`, и тогда точка будет ловить вообще любой символ, включая новую строку. Эти флаги слегка меняют поведение функции, вот и все.